In [1]:
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

In [3]:
from utils_training import Dataset_LV
from utils_training import LVTrainingData
from utils_training import evaluate_LVmodel

In [4]:
from behavior_model import LVBehaviorModel

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'Synthetic_crash_scenarios.csv'
# file_path = '/content/drive/MyDrive/Colab Notebooks/Thesis Crash Generation/Synthetic_crash_scenarios.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

## Build the dataset

In [7]:
acc_min, acc_max, acc_interval =  -9.5, 7, 0.25
acc_std_dev = 0.07
Inputs, Outputs, Weights = LVTrainingData(df)

In [8]:
## Load the train, val, test indexes

import pickle
with open('.\\utils_data\\Idx_02_split.pkl', 'rb') as f:
    list_train_val_test = pickle.load(f)
list_train_val_test = list_train_val_test[0]

Idx_train, Idx_val, Idx_test = list_train_val_test[0], list_train_val_test[1], list_train_val_test[2]

In [9]:
## Split the Inputs, Outputs, Weights based on train, val, test indexes

Inputs_train = [Inputs[i] for i in Idx_train]
Outputs_train = [Outputs[i] for i in Idx_train]
Weights_train = [Weights[i] for i in Idx_train]

Inputs_val = [Inputs[i] for i in Idx_val]
Outputs_val = [Outputs[i] for i in Idx_val]
Weights_val = [Weights[i] for i in Idx_val]

In [10]:
## Create dataset and dataloader

input_size = Inputs[0].shape[1] # Dimension of the input
batch_size = 64

# dataset = BCDataset(Inputs, Outputs, Weights)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_train = Dataset_LV(Inputs_train, Outputs_train, Weights_train)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = Dataset_LV(Inputs_val, Outputs_val, Weights_val)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

## Model Training

In [11]:
## Network Parameters 
gru_hidden_size = 256 # Size of GRU's hidden layer
mlp_hidden_size = 128 # Size of MLP's hidden layer
output_size = int((acc_max - acc_min)/acc_interval) + 1

## Training Parameters
num_epochs = 500
learning_rate = 0.0001

## Instantiate model, loss function, and optimizer
model = LVBehaviorModel(input_size, gru_hidden_size, mlp_hidden_size, output_size).to(device)
# criterion = nn.KLDivLoss(reduction='batchmean').to(device)
criterion = nn.KLDivLoss(reduction='none').to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [12]:
best_val_loss = float('inf')
best_train_loss = float('inf')
best_trainval_loss = float('inf')
best_val_epoch = 0
best_train_epoch = 0
best_trainval_epoch = 0
best_val_model_path = f'.\\weights\\bc_lv_val_02.pth'
best_train_model_path = f'.\\weights\\bc_lv_train_02.pth'
best_trainval_model_path = f'.\\weights\\bc_lv_trainval_02.pth'

In [13]:
num_epochs = 4000

In [14]:
for epoch in range(num_epochs):
    model.train() # set model as training mode
    for inputs, targets, sample_weights in dataloader_train:
        inputs = inputs.float().to(device)
        targets = targets.float().to(device)
        sample_weights = sample_weights.to(device)
        
        optimizer.zero_grad()
        log_probs = model(inputs)
        ## calculate loss
        loss = criterion(log_probs, targets)
        weighted_loss = torch.sum(torch.sum(loss,axis=2) * sample_weights.reshape(-1,1))/torch.sum(sample_weights) ## loss per weight
        weighted_loss.backward()
        optimizer.step()
    
    ## Calculate the mean, min, max loss of this epoch
    model.eval()  # set model as evaluation mode
    ## loss of training dataset
    train_loss_mean, train_loss_min, train_loss_max = evaluate_LVmodel(model, criterion, dataloader_train, device)
    ## loss of validation dataset
    test_loss_mean, test_loss_min, test_loss_max = evaluate_LVmodel(model, criterion, dataloader_val, device)
    ## loss of combination
    trainval_loss_mean = (test_loss_mean*len(dataset_val) + train_loss_mean*len(dataset_train)) / (len(dataset_val)+len(dataset_train))

    print(f'Epoch {epoch+1}/{num_epochs}, Loss_mean: {round(train_loss_mean.item(),2)}, \
     Loss_min: {round(train_loss_min.item(),2)}, Loss_max: {round(train_loss_max.item(),2)} \n \
     Loss_mean_val: {round(test_loss_mean.item(),2)}, Loss_min_val: {round(test_loss_min.item(),2)}, \
     Loss_max_val: {round(test_loss_max.item(),2)} \n \
     Loss_mean_trainval: {round(trainval_loss_mean.item(),2)}')

    # Save the model if the current test loss is the best seen so far
    if test_loss_mean < best_val_loss:
        best_val_epoch = epoch+1
        best_val_loss = test_loss_mean
        best_val_model = model.state_dict()
    if train_loss_mean < best_train_loss:
        best_train_epoch = epoch+1
        best_train_loss = train_loss_mean
        best_train_model = model.state_dict()
    if trainval_loss_mean < best_trainval_loss:
        best_trainval_epoch = epoch+1
        best_trainval_loss = trainval_loss_mean
        best_trainval_model = model.state_dict()
        
torch.save(best_val_model, best_val_model_path)
print(f"Model saved with the best val loss. \n Epoch: {best_val_epoch}, Loss_val: {round(best_val_loss.item(),2)}")
torch.save(best_train_model, best_train_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_train_epoch}, Loss_val: {round(best_train_loss.item(),2)}")
torch.save(best_trainval_model, best_trainval_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_trainval_epoch}, Loss_val: {round(best_trainval_loss.item(),2)}")

Epoch 1/4000, Loss_mean: 396.17,      Loss_min: 390.37, Loss_max: 406.42 
      Loss_mean_val: 390.76, Loss_min_val: 384.61,      Loss_max_val: 400.41 
      Loss_mean_trainval: 391.02
Epoch 2/4000, Loss_mean: 384.73,      Loss_min: 377.74, Loss_max: 391.21 
      Loss_mean_val: 388.52, Loss_min_val: 380.24,      Loss_max_val: 396.07 
      Loss_mean_trainval: 388.34
Epoch 3/4000, Loss_mean: 386.24,      Loss_min: 383.59, Loss_max: 387.8 
      Loss_mean_val: 386.54, Loss_min_val: 373.94,      Loss_max_val: 394.76 
      Loss_mean_trainval: 386.53
Epoch 4/4000, Loss_mean: 384.09,      Loss_min: 382.48, Loss_max: 385.64 
      Loss_mean_val: 384.66, Loss_min_val: 377.58,      Loss_max_val: 390.76 
      Loss_mean_trainval: 384.63
Epoch 5/4000, Loss_mean: 381.21,      Loss_min: 377.73, Loss_max: 385.25 
      Loss_mean_val: 382.89, Loss_min_val: 374.92,      Loss_max_val: 389.01 
      Loss_mean_trainval: 382.81
Epoch 6/4000, Loss_mean: 388.45,      Loss_min: 378.5, Loss_max: 406.98 
   

Epoch 46/4000, Loss_mean: 232.19,      Loss_min: 220.45, Loss_max: 253.48 
      Loss_mean_val: 245.94, Loss_min_val: 181.48,      Loss_max_val: 307.37 
      Loss_mean_trainval: 245.28
Epoch 47/4000, Loss_mean: 241.62,      Loss_min: 181.01, Loss_max: 280.12 
      Loss_mean_val: 243.15, Loss_min_val: 161.68,      Loss_max_val: 299.84 
      Loss_mean_trainval: 243.08
Epoch 48/4000, Loss_mean: 273.82,      Loss_min: 214.83, Loss_max: 365.38 
      Loss_mean_val: 241.75, Loss_min_val: 162.11,      Loss_max_val: 328.21 
      Loss_mean_trainval: 243.28
Epoch 49/4000, Loss_mean: 251.76,      Loss_min: 227.39, Loss_max: 288.34 
      Loss_mean_val: 241.68, Loss_min_val: 170.61,      Loss_max_val: 332.89 
      Loss_mean_trainval: 242.16
Epoch 50/4000, Loss_mean: 255.45,      Loss_min: 224.48, Loss_max: 305.86 
      Loss_mean_val: 240.88, Loss_min_val: 117.65,      Loss_max_val: 304.43 
      Loss_mean_trainval: 241.57
Epoch 51/4000, Loss_mean: 238.98,      Loss_min: 225.5, Loss_max: 256.

Epoch 92/4000, Loss_mean: 241.17,      Loss_min: 191.06, Loss_max: 322.93 
      Loss_mean_val: 222.8, Loss_min_val: 124.71,      Loss_max_val: 271.51 
      Loss_mean_trainval: 223.67
Epoch 93/4000, Loss_mean: 235.03,      Loss_min: 184.26, Loss_max: 307.82 
      Loss_mean_val: 223.75, Loss_min_val: 105.85,      Loss_max_val: 294.12 
      Loss_mean_trainval: 224.29
Epoch 94/4000, Loss_mean: 226.75,      Loss_min: 170.66, Loss_max: 255.88 
      Loss_mean_val: 224.06, Loss_min_val: 133.34,      Loss_max_val: 291.4 
      Loss_mean_trainval: 224.19
Epoch 95/4000, Loss_mean: 237.73,      Loss_min: 184.78, Loss_max: 303.05 
      Loss_mean_val: 222.81, Loss_min_val: 150.32,      Loss_max_val: 281.15 
      Loss_mean_trainval: 223.52
Epoch 96/4000, Loss_mean: 165.98,      Loss_min: 97.03, Loss_max: 224.35 
      Loss_mean_val: 226.1, Loss_min_val: 110.17,      Loss_max_val: 308.42 
      Loss_mean_trainval: 223.24
Epoch 97/4000, Loss_mean: 197.36,      Loss_min: 168.19, Loss_max: 226.8 


Epoch 137/4000, Loss_mean: 162.4,      Loss_min: 119.78, Loss_max: 184.78 
      Loss_mean_val: 212.62, Loss_min_val: 106.71,      Loss_max_val: 273.4 
      Loss_mean_trainval: 210.22
Epoch 138/4000, Loss_mean: 211.46,      Loss_min: 158.74, Loss_max: 274.11 
      Loss_mean_val: 213.79, Loss_min_val: 101.46,      Loss_max_val: 289.76 
      Loss_mean_trainval: 213.68
Epoch 139/4000, Loss_mean: 200.96,      Loss_min: 155.82, Loss_max: 237.88 
      Loss_mean_val: 211.69, Loss_min_val: 117.1,      Loss_max_val: 259.39 
      Loss_mean_trainval: 211.18
Epoch 140/4000, Loss_mean: 205.47,      Loss_min: 165.91, Loss_max: 253.87 
      Loss_mean_val: 211.67, Loss_min_val: 145.62,      Loss_max_val: 268.65 
      Loss_mean_trainval: 211.37
Epoch 141/4000, Loss_mean: 222.15,      Loss_min: 161.75, Loss_max: 314.47 
      Loss_mean_val: 209.46, Loss_min_val: 136.44,      Loss_max_val: 301.85 
      Loss_mean_trainval: 210.06
Epoch 142/4000, Loss_mean: 195.04,      Loss_min: 172.0, Loss_max: 2

Epoch 182/4000, Loss_mean: 256.18,      Loss_min: 173.09, Loss_max: 420.09 
      Loss_mean_val: 206.58, Loss_min_val: 122.75,      Loss_max_val: 287.99 
      Loss_mean_trainval: 208.94
Epoch 183/4000, Loss_mean: 183.77,      Loss_min: 138.04, Loss_max: 207.0 
      Loss_mean_val: 206.16, Loss_min_val: 122.12,      Loss_max_val: 277.7 
      Loss_mean_trainval: 205.09
Epoch 184/4000, Loss_mean: 205.23,      Loss_min: 165.17, Loss_max: 277.73 
      Loss_mean_val: 205.24, Loss_min_val: 109.34,      Loss_max_val: 266.6 
      Loss_mean_trainval: 205.24
Epoch 185/4000, Loss_mean: 192.86,      Loss_min: 160.98, Loss_max: 242.52 
      Loss_mean_val: 204.94, Loss_min_val: 96.89,      Loss_max_val: 269.83 
      Loss_mean_trainval: 204.37
Epoch 186/4000, Loss_mean: 196.71,      Loss_min: 156.77, Loss_max: 249.01 
      Loss_mean_val: 205.53, Loss_min_val: 109.07,      Loss_max_val: 255.88 
      Loss_mean_trainval: 205.11
Epoch 187/4000, Loss_mean: 144.89,      Loss_min: 99.95, Loss_max: 18

...

Epoch 3782/4000, Loss_mean: 18.36,      Loss_min: 15.53, Loss_max: 22.98 
      Loss_mean_val: 158.19, Loss_min_val: 87.68,      Loss_max_val: 385.93 
      Loss_mean_trainval: 151.52
Epoch 3783/4000, Loss_mean: 38.58,      Loss_min: 15.32, Loss_max: 83.25 
      Loss_mean_val: 156.94, Loss_min_val: 85.21,      Loss_max_val: 383.42 
      Loss_mean_trainval: 151.3
Epoch 3784/4000, Loss_mean: 12.62,      Loss_min: 2.69, Loss_max: 20.74 
      Loss_mean_val: 156.89, Loss_min_val: 43.14,      Loss_max_val: 425.63 
      Loss_mean_trainval: 150.01
Epoch 3785/4000, Loss_mean: 17.96,      Loss_min: 14.82, Loss_max: 21.76 
      Loss_mean_val: 151.46, Loss_min_val: 75.68,      Loss_max_val: 486.47 
      Loss_mean_trainval: 145.09
Epoch 3786/4000, Loss_mean: 13.81,      Loss_min: 7.91, Loss_max: 17.35 
      Loss_mean_val: 158.86, Loss_min_val: 65.81,      Loss_max_val: 352.85 
      Loss_mean_trainval: 151.94
Epoch 3787/4000, Loss_mean: 41.9,      Loss_min: 13.0, Loss_max: 93.83 
      Loss_

Epoch 3827/4000, Loss_mean: 19.5,      Loss_min: 15.21, Loss_max: 26.42 
      Loss_mean_val: 158.8, Loss_min_val: 54.83,      Loss_max_val: 465.57 
      Loss_mean_trainval: 152.15
Epoch 3828/4000, Loss_mean: 20.96,      Loss_min: 13.34, Loss_max: 29.76 
      Loss_mean_val: 160.92, Loss_min_val: 90.39,      Loss_max_val: 439.44 
      Loss_mean_trainval: 154.25
Epoch 3829/4000, Loss_mean: 26.39,      Loss_min: 11.7, Loss_max: 44.91 
      Loss_mean_val: 161.52, Loss_min_val: 50.57,      Loss_max_val: 416.91 
      Loss_mean_trainval: 155.08
Epoch 3830/4000, Loss_mean: 20.44,      Loss_min: 14.51, Loss_max: 29.62 
      Loss_mean_val: 160.61, Loss_min_val: 64.63,      Loss_max_val: 347.43 
      Loss_mean_trainval: 153.92
Epoch 3831/4000, Loss_mean: 19.28,      Loss_min: 15.25, Loss_max: 26.06 
      Loss_mean_val: 164.38, Loss_min_val: 72.35,      Loss_max_val: 429.35 
      Loss_mean_trainval: 157.46
Epoch 3832/4000, Loss_mean: 16.71,      Loss_min: 13.14, Loss_max: 20.84 
      Los

Epoch 3872/4000, Loss_mean: 13.19,      Loss_min: 6.19, Loss_max: 20.14 
      Loss_mean_val: 160.43, Loss_min_val: 85.18,      Loss_max_val: 433.9 
      Loss_mean_trainval: 153.41
Epoch 3873/4000, Loss_mean: 15.59,      Loss_min: 14.41, Loss_max: 16.84 
      Loss_mean_val: 161.0, Loss_min_val: 63.69,      Loss_max_val: 300.76 
      Loss_mean_trainval: 154.06
Epoch 3874/4000, Loss_mean: 13.26,      Loss_min: 1.85, Loss_max: 19.37 
      Loss_mean_val: 167.94, Loss_min_val: 33.45,      Loss_max_val: 405.35 
      Loss_mean_trainval: 160.56
Epoch 3875/4000, Loss_mean: 26.14,      Loss_min: 19.46, Loss_max: 37.84 
      Loss_mean_val: 174.33, Loss_min_val: 73.53,      Loss_max_val: 398.66 
      Loss_mean_trainval: 167.27
Epoch 3876/4000, Loss_mean: 17.98,      Loss_min: 13.52, Loss_max: 21.47 
      Loss_mean_val: 170.53, Loss_min_val: 89.13,      Loss_max_val: 451.54 
      Loss_mean_trainval: 163.26
Epoch 3877/4000, Loss_mean: 48.33,      Loss_min: 15.61, Loss_max: 108.68 
      Los

Epoch 3917/4000, Loss_mean: 13.68,      Loss_min: 6.51, Loss_max: 19.63 
      Loss_mean_val: 155.62, Loss_min_val: 63.0,      Loss_max_val: 360.93 
      Loss_mean_trainval: 148.85
Epoch 3918/4000, Loss_mean: 11.98,      Loss_min: 0.66, Loss_max: 21.09 
      Loss_mean_val: 162.31, Loss_min_val: 63.95,      Loss_max_val: 469.6 
      Loss_mean_trainval: 155.14
Epoch 3919/4000, Loss_mean: 20.21,      Loss_min: 17.19, Loss_max: 25.68 
      Loss_mean_val: 161.76, Loss_min_val: 97.36,      Loss_max_val: 488.62 
      Loss_mean_trainval: 155.01
Epoch 3920/4000, Loss_mean: 14.15,      Loss_min: 7.34, Loss_max: 18.12 
      Loss_mean_val: 157.25, Loss_min_val: 99.77,      Loss_max_val: 301.87 
      Loss_mean_trainval: 150.43
Epoch 3921/4000, Loss_mean: 35.2,      Loss_min: 16.05, Loss_max: 72.33 
      Loss_mean_val: 159.7, Loss_min_val: 57.45,      Loss_max_val: 364.56 
      Loss_mean_trainval: 153.76
Epoch 3922/4000, Loss_mean: 12.62,      Loss_min: 1.62, Loss_max: 24.2 
      Loss_mean

Epoch 3962/4000, Loss_mean: 24.95,      Loss_min: 14.64, Loss_max: 43.54 
      Loss_mean_val: 151.43, Loss_min_val: 55.18,      Loss_max_val: 370.93 
      Loss_mean_trainval: 145.4
Epoch 3963/4000, Loss_mean: 19.46,      Loss_min: 14.38, Loss_max: 26.97 
      Loss_mean_val: 155.91, Loss_min_val: 66.89,      Loss_max_val: 299.62 
      Loss_mean_trainval: 149.4
Epoch 3964/4000, Loss_mean: 34.38,      Loss_min: 15.13, Loss_max: 71.13 
      Loss_mean_val: 155.7, Loss_min_val: 63.92,      Loss_max_val: 334.66 
      Loss_mean_trainval: 149.91
Epoch 3965/4000, Loss_mean: 12.2,      Loss_min: 2.39, Loss_max: 21.35 
      Loss_mean_val: 157.15, Loss_min_val: 82.78,      Loss_max_val: 380.86 
      Loss_mean_trainval: 150.24
Epoch 3966/4000, Loss_mean: 30.85,      Loss_min: 14.55, Loss_max: 60.14 
      Loss_mean_val: 159.07, Loss_min_val: 73.85,      Loss_max_val: 389.53 
      Loss_mean_trainval: 152.96
Epoch 3967/4000, Loss_mean: 15.23,      Loss_min: 12.25, Loss_max: 18.65 
      Loss_